# Effect of including average bonus + save + card points

In [3]:
import pandas as pd
from sqlalchemy import create_engine

from airsenal.framework.prediction_utils import (
    fit_bonus_points,
    fit_card_points,
    fit_def_con,
    fit_save_points,
)
from airsenal.framework.schema import Player, get_connection_string, session
from airsenal.framework.season import CURRENT_SEASON
from airsenal.framework.utils import get_player, list_players

engine = create_engine(get_connection_string())

In [4]:
df_bonus = fit_bonus_points()[0]
df_cards = fit_card_points()
df_saves = fit_save_points()
df_def_con = fit_def_con()[0]

df = pd.DataFrame(
    {"bonus": df_bonus, "cards": df_cards, "saves": df_saves, "def_con": df_def_con}
)

df.fillna(0, inplace=True)
df["TOTAL"] = df.sum(axis=1)

players = pd.read_sql(session.query(Player).statement, engine)

df = pd.merge(df, players, how="left", left_on="player_id", right_on="player_id")

current_players = list_players()
current_ids = [p.player_id for p in current_players]
current_positions = pd.Series(
    {p.player_id: p.position(CURRENT_SEASON) for p in current_players}, name="position"
)
df = df[df.player_id.isin(current_ids)]
df = pd.merge(df, current_positions, how="left", left_on="player_id", right_index=True)
df.set_index(["player_id", "name"], inplace=True)

## Gain most overall

In [5]:
df.sort_values(by="TOTAL", ascending=False).head(20)

,,bonus,cards,saves,def_con,TOTAL,fpl_api_id,display_name,opta_code,position
player_id,name,,,,,,,,,
125,Marcos Senesi Barón,0.300325,-0.303788,0.000000,1.276495,1.273032,72.0,Senesi,p221466,DEF
318,James Tarkowski,0.280218,-0.201537,0.000000,1.174105,1.252786,291.0,Tarkowski,p17761,DEF
311,Jaydee Canvot,0.519213,-0.078704,0.000000,0.797612,1.238121,723.0,Canvot,p651426,DEF
680,Wes Foderingham,0.247551,-0.085417,1.070328,0.000000,1.232463,601.0,Foderingham,p82143,GK
597,Daniel Ballard,0.401328,-0.122024,0.000000,0.937885,1.217189,531.0,Ballard,p223827,DEF
679,Alphonse Areola,0.198268,-0.026852,0.984853,0.000000,1.156269,600.0,Areola,p84182,GK
574,Elliot Anderson,0.266192,-0.164435,0.000000,1.047641,1.149399,517.0,Anderson,p215379,MID
674,João Maria Lobo Alves Palhares Costa Palhinha Gonçalves,0.634746,-0.213889,0.000000,0.705659,1.126517,673.0,J.Palhinha,p154296,MID
166,Nathan Collins,0.295091,-0.143880,0.000000,0.953914,1.105125,106.0,Collins,p432830,DEF


## Lose most overall

In [6]:
df.sort_values(by="TOTAL", ascending=True).head(20)

,,bonus,cards,saves,def_con,TOTAL,fpl_api_id,display_name,opta_code,position
player_id,name,,,,,,,,,
605,Luke O'Nien,0.000000,-0.339744,0.0,0.0,-0.339744,539.0,O'Nien,p167512,DEF
695,Edson Álvarez Velázquez,0.114700,-0.324879,0.0,0.0,-0.210179,616.0,Álvarez,p213999,MID
661,Yves Bissouma,0.128971,-0.315741,0.0,0.0,-0.186770,587.0,Bissouma,p227127,MID
441,Jayden Danns,0.000000,-0.185897,0.0,0.0,-0.185897,398.0,Danns,p500058,FWD
633,Arthur Masuaku,0.000000,-0.185897,0.0,0.0,-0.185897,678.0,Masuaku,p105717,DEF
81,Connor Roberts,0.123920,-0.309028,0.0,0.0,-0.185108,186.0,Roberts,p192290,DEF
550,Joe White,0.000000,-0.172619,0.0,0.0,-0.172619,498.0,White,p461484,MID
264,Roméo Lavia,0.068940,-0.240278,0.0,0.0,-0.171338,244.0,Lavia,p514356,MID
144,Romain Faivre,0.000000,-0.161111,0.0,0.0,-0.161111,92.0,Faivre,p441240,MID


## Top GK

In [7]:
df[df.position == "GK"].sort_values(by="TOTAL", ascending=False).head(20)

,,bonus,cards,saves,def_con,TOTAL,fpl_api_id,display_name,opta_code,position
player_id,name,,,,,,,,,
680,Wes Foderingham,0.247551,-0.085417,1.070328,0.0,1.232463,601.0,Foderingham,p82143,GK
679,Alphonse Areola,0.198268,-0.026852,0.984853,0.0,1.156269,600.0,Areola,p84182,GK
632,Robin Roefs,0.364203,-0.106771,0.804462,0.0,1.061895,670.0,Roefs,p498016,GK
119,Norberto Murara Neto,0.366493,-0.146714,0.839183,0.0,1.058962,66.0,Neto,p69752,GK
115,Martin Dúbravka,0.253679,-0.033730,0.827209,0.0,1.047158,470.0,Dúbravka,p67089,GK
711,Lukasz Fabianski,0.267735,-0.043155,0.794856,0.0,1.019436,738.0,Fabianski,p37096,GK
314,Jordan Pickford,0.418307,-0.133908,0.708571,0.0,0.992970,287.0,Pickford,p111234,GK
2,Kepa Arrizabalaga Revuelta,0.357312,-0.077381,0.706417,0.0,0.986348,2.0,Arrizabalaga,p109745,GK
447,James Trafford,0.186695,-0.083333,0.871775,0.0,0.975137,182.0,Trafford,p432720,GK


## Top DEF

In [8]:
df[df.position == "DEF"].sort_values(by="TOTAL", ascending=False).head(20)

,,bonus,cards,saves,def_con,TOTAL,fpl_api_id,display_name,opta_code,position
player_id,name,,,,,,,,,
125,Marcos Senesi Barón,0.300325,-0.303788,0.0,1.276495,1.273032,72.0,Senesi,p221466,DEF
318,James Tarkowski,0.280218,-0.201537,0.0,1.174105,1.252786,291.0,Tarkowski,p17761,DEF
311,Jaydee Canvot,0.519213,-0.078704,0.0,0.797612,1.238121,723.0,Canvot,p651426,DEF
597,Daniel Ballard,0.401328,-0.122024,0.0,0.937885,1.217189,531.0,Ballard,p223827,DEF
166,Nathan Collins,0.295091,-0.143880,0.0,0.953914,1.105125,106.0,Collins,p432830,DEF
322,Michael Keane,0.386010,-0.178385,0.0,0.881771,1.089395,295.0,Keane,p106611,DEF
635,Nordi Mukiele,0.341018,-0.180556,0.0,0.919045,1.079508,694.0,Mukiele,p200834,DEF
685,Konstantinos Mavropanos,0.164831,-0.161255,0.0,1.071425,1.075001,606.0,Mavropanos,p233963,DEF
554,Malick Thiaw,0.452983,-0.152299,0.0,0.724865,1.025549,684.0,Thiaw,p465642,DEF


## Top MID

In [9]:
df[df.position == "MID"].sort_values(by="TOTAL", ascending=False).head(20)

,,bonus,cards,saves,def_con,TOTAL,fpl_api_id,display_name,opta_code,position
player_id,name,,,,,,,,,
574,Elliot Anderson,0.266192,-0.164435,0.0,1.047641,1.149399,517.0,Anderson,p215379,MID
674,João Maria Lobo Alves Palhares Costa Palhinha Gonçalves,0.634746,-0.213889,0.0,0.705659,1.126517,673.0,J.Palhinha,p154296,MID
466,Rayan Cherki,0.775801,-0.089506,0.0,0.418028,1.104323,417.0,Cherki,p466052,MID
501,Bruno Borges Fernandes,0.749730,-0.200122,0.0,0.483018,1.032627,449.0,B.Fernandes,p141746,MID
20,Declan Rice,0.488454,-0.144799,0.0,0.650817,0.994472,21.0,Rice,p204480,MID
256,Cole Palmer,0.832910,-0.172118,0.0,0.324971,0.985762,235.0,Palmer,p244851,MID
426,Mohamed Salah,0.863715,-0.056187,0.0,0.153239,0.960768,381.0,M.Salah,p118748,MID
462,Omar Marmoush,0.678577,-0.039352,0.0,0.320410,0.959634,413.0,Marmoush,p438234,MID
15,Bukayo Saka,0.693373,-0.120443,0.0,0.380980,0.953911,16.0,Saka,p223340,MID


## Top FWD

In [10]:
df[df.position == "FWD"].sort_values(by="TOTAL", ascending=False).head(20)

,,bonus,cards,saves,def_con,TOTAL,fpl_api_id,display_name,opta_code,position
player_id,name,,,,,,,,,
477,Erling Haaland,1.047249,-0.072997,0.0,0.005952,0.980204,430.0,Haaland,p223094,FWD
443,Hugo Ekitiké,0.946278,-0.048851,0.0,0.008282,0.905709,661.0,Ekitiké,p510663,FWD
582,Chris Wood,0.908410,-0.021970,0.0,0.011905,0.898345,525.0,Wood,p60689,FWD
442,Alexander Isak,0.876532,-0.060535,0.0,0.011905,0.827902,499.0,Isak,p219168,FWD
1031,Mateus Mané,0.784293,-0.064394,0.0,0.012698,0.732598,749.0,None,p647671,FWD
269,João Pedro Junqueira de Jesus,0.796634,-0.137963,0.0,0.006803,0.665474,249.0,João Pedro,p475168,FWD
67,Ollie Watkins,0.736649,-0.086227,0.0,0.006568,0.656991,64.0,Watkins,p178301,FWD
519,Benjamin Sesko,0.724442,-0.089506,0.0,0.010025,0.644961,681.0,Šeško,p485711,FWD
703,Jarrod Bowen,0.490779,-0.068237,0.0,0.193452,0.615995,624.0,Bowen,p178186,FWD


## Top Bonus

In [20]:
df.sort_values(by="bonus", ascending=False)[["bonus"]].head(20)

,,bonus
player_id,name,
477,Erling Haaland,1.047249
443,Hugo Ekitiké,0.946278
582,Chris Wood,0.908410
442,Alexander Isak,0.876532
426,Mohamed Salah,0.863715
256,Cole Palmer,0.832910
269,João Pedro Junqueira de Jesus,0.796634
1031,Mateus Mané,0.784293
466,Rayan Cherki,0.775801


## Top Def Con

In [21]:
df.sort_values(by="def_con", ascending=False)[["def_con"]].head(20)

,,def_con
player_id,name,
125,Marcos Senesi Barón,1.276495
318,James Tarkowski,1.174105
685,Konstantinos Mavropanos,1.071425
351,Joachim Andersen,1.052378
574,Elliot Anderson,1.047641
166,Nathan Collins,0.953914
597,Daniel Ballard,0.937885
280,Maxence Lacroix,0.924105
635,Nordi Mukiele,0.919045


## Top Cards

In [22]:
df.sort_values(by="cards")[["cards"]].head(20)

,,cards
player_id,name,
719,Yerson Mosquera Valdelamar,-0.372024
607,Reinildo Mandava,-0.365942
643,Cristian Romero,-0.352987
605,Luke O'Nien,-0.339744
695,Edson Álvarez Velázquez,-0.324879
507,Carlos Henrique Casimiro,-0.321651
731,João Victor Gomes da Silva,-0.316220
543,Joelinton Cássio Apolinário de Lira,-0.315749
661,Yves Bissouma,-0.315741


## Top Saves

In [24]:
df.sort_values(by="saves", ascending=False)[["saves"]].head(20)

,,saves
player_id,name,
680,Wes Foderingham,1.070328
679,Alphonse Areola,0.984853
447,James Trafford,0.871775
119,Norberto Murara Neto,0.839183
115,Martin Dúbravka,0.827209
632,Robin Roefs,0.804462
641,Antonín Kinský,0.796425
711,Lukasz Fabianski,0.794856
315,Mark Travers,0.791426


## Specific player

In [11]:
p = get_player("Haaland").player_id
df[df.index.get_level_values(0) == p]

,,bonus,cards,saves,def_con,TOTAL,fpl_api_id,display_name,opta_code,position
player_id,name,,,,,,,,,
477,Erling Haaland,1.047249,-0.072997,0.0,0.005952,0.980204,430.0,Haaland,p223094,FWD
